# 🏥 MQ10: Early Stopping & Regularization with Medical Data

### 🧩 Problem Statement
- **What**: Train an AI to detect breast cancer from medical data without "memorizing" the answers.
- **Why**: If a model simply memorizes the training data (overfitting), it will fail when it sees a new patient. In medicine, this could mean missing a diagnosis.
- **Real-World Relevance**: Medical AI systems MUST be robust. They cannot afford to only work on data they've seen before.

### 🪜 Steps to Solve the Problem
1.  **Import Libraries**: Get the tools we need (TensorFlow, Scikit-Learn).
2.  **Load & Preprocess**: Get the data, split it into Practice (Train), Mock Exam (Val), and Final Exam (Test). Standardize it.
3.  **Build Model**: Create a brain with 2 hidden layers.
4.  **Add Protections**: Add **Weight Decay** (Simplicity Rule) and **Early Stopping** (Stop when not learning).
5.  **Train**: Run the training loop.
6.  **Evaluate**: See the results.

### 🎯 Expected Output
- Ideally, the training will **stop early** (e.g., at epoch 30 instead of 100).
- The Validation AUC (Accuracy) should be high (> 0.95).
- We will see a graph showing where it stopped.

### 🔹 Import Libraries and Set Seeds
#### 2.1 What the line does
Imports necessary libraries and sets random seeds for reproducibility.
#### 2.2 Why it is used
To ensure that every time we run this notebook, we get the exact same results. This is crucial for teaching and debugging.
#### 2.3 When to use it
At the start of EVERY machine learning project.
#### 2.4 Where to use it
First code cell.
#### 2.5 How to use it
`random.seed(42)`
#### 2.6 How it works internally
Computer random numbers aren't real magic; they are math sequences. Setting the seed sets the starting number of that sequence.
#### 2.7 Output with sample examples
No visible output, but internally the random generators are fixed.

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models, regularizers, callbacks
import matplotlib.pyplot as plt

# Set Random Seeds
os.environ['PYTHONHASHSEED'] = '0'
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

### 🔹 Load and Split Data
#### 2.1 What the line does
Loads the Breast Cancer dataset and splits it into **Train**, **Validation**, and **Test** sets.
#### 2.2 Why it is used
We need three separate buckets of data:
- **Train (70%)**: To teach the model.
- **Validation (15%)**: To check progress during training (Mock Exam).
- **Test (15%)**: Final unbiased evaluation (Final Exam).
#### 2.3 When to use it
Before any training begins.
#### 2.4 Where to use it
Data preparation stage.
#### 2.5 How to use it
We use `train_test_split` twice.
#### 2.6 How it works internally
It shuffles the data and then cuts it based on the percentage (`test_size`). The `stratify=y` ensures we keep the same ratio of Sick/Healthy patients in all buckets.
#### 2.7 Output with sample examples
Arrays of numbers representing the data split.

In [ ]:
# Load Data
data = load_breast_cancer()
X = data.data
y = data.target

# 1st Split: Train (70%) vs Temp (30%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# 2nd Split: Validation (15%) vs Test (15%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

### 🔹 Standardize Features
#### 2.1 What the line does
Scales all features so they have Mean = 0 and Standard Deviation = 1.
#### 2.2 Why it is used
Some features are big numbers (Area = 1000), some are small (Symmetry = 0.1). Neural networks get confused by big mismatches. Scaling makes them all technically equal.
#### 2.3 When to use it
For almost all neural networks using tabular data.
#### 2.4 Where to use it
After splitting the data.
#### 2.5 How to use it
`scaler.fit_transform(X_train)`
#### 2.6 How it works internally
Calculates average and spread of the training data, then subtracts average and divides by spread for every number.
#### 2.7 Output with sample examples
All numbers will be roughly between -3 and +3.

In [ ]:
scaler = StandardScaler()

# Fit ONLY on Training data, then transform all
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(f"Training Data Shape: {X_train_scaled.shape}")

### 🔹 Build the Model (with L2 Regularization)
#### 3.1 What it does
Creates a neural network with two hidden layers (64 & 32 neurons).
#### 3.2 Why it is used
We add `kernel_regularizer=regularizers.l2(1e-4)`. This is **Weight Decay**.
- It penalizes the model for having "large" weights.
- Large weights usually mean the model is memorizing noise (Overfitting).
- Small weights mean the model is looking for smoother, simpler patterns.
#### 3.3 When to use it
When you have a small dataset (like this one) and want to prevent overfitting.
#### 3.4 Where to use it
Inside the `Dense` layer definition.
#### 3.5 How to use it
`kernel_regularizer=regularizers.l2(0.0001)`
#### 3.6 How it affects execution internally
It adds a small penalty term to the Loss function: `Loss = Error + (0.0001 * Weights^2)`.
#### 3.7 Output impact with examples
The model learns slightly slower but generalizes better.

In [ ]:
model = models.Sequential()

# Hidden Layer 1: 64 Neurons, ReLU, L2 Regularization
model.add(layers.Dense(
    64, 
    activation='relu', 
    input_shape=(X_train_scaled.shape[1],),
    kernel_regularizer=regularizers.l2(1e-4)
))

# Hidden Layer 2: 32 Neurons, ReLU, L2 Regularization
model.add(layers.Dense(
    32, 
    activation='relu',
    kernel_regularizer=regularizers.l2(1e-4)
))

# Output Layer: 1 Neuron (Sigmoid for binary class)
model.add(layers.Dense(1, activation='sigmoid'))

# Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['AUC', 'accuracy']
)

### 🔹 Define Early Stopping
#### 3.1 What it does
Creates a watcher that stops training if the Validation Loss stops improving.
#### 3.2 Why it is used
To save time and prevent overfitting. If the model stops learning, why keep teaching it?
#### 3.3 When to use it
Always with iterative training.
#### 3.4 Where to use it
Defined before `model.fit()`.
#### 3.5 How to use it (Arguments)
- `monitor='val_loss'`: Watch the mock exam score.
- `patience=4`: If it doesn't improve for 4 epochs in a row, STOP.
- `restore_best_weights=True`: Go back in time to the moment it was best.
#### 3.6 How it affects execution internally
Checks the metric after every epoch.
#### 3.7 Output impact with examples
Training might stop at epoch 35 instead of completing 100.

In [ ]:
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=4,
    restore_best_weights=True,
    verbose=1
)

### 🔹 Train the Model
#### 2.1 What the line does
Starts the training loop where the model learns from data.
#### 2.2 Why it is used
This is the actual "learning" phase.
#### 2.3 When to use it
After compiling.
#### 2.4 Where to use it
It returns a `history` object we can plot.
#### 2.5 How to use it
Pass the training data, validation data, and the `callbacks` list.
#### 2.6 How it works internally
Iterates through data, calculates error, updates weights using Adam, repeats.
#### 2.7 Output with sample examples
A progress bar showing Loss/AUC for every epoch.

In [ ]:
print("Starting training...")
history = model.fit(
    X_train_scaled, y_train,
    epochs=100,  # We set this high, counting on Early Stopping to stop it
    batch_size=32,
    validation_data=(X_val_scaled, y_val),
    callbacks=[early_stopping],
    verbose=1
)

### 🔹 Evaluate and Visualize
#### 2.1 What the line does
Plots the training history and calculates final scores on the Test set.
#### 2.2 Why it is used
To verify if the model actually generalized well.
#### 2.3 When to use it
After training stops.
#### 2.4 Where to use it
Final analysis.
#### 2.5 How to use it
Use Matplotlib to draw lines for Loss and AUC.
#### 2.6 How it works internally
Reads the `history.history` dictionary.
#### 2.7 Output with sample examples
A graph showing the lines going down (Loss) and up (AUC).

In [ ]:
# Final Evaluation
test_loss, test_auc, test_acc = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"\nTest Accuracy: {test_acc:.4f}")
print(f"Test AUC: {test_auc:.4f}")
print(f"Stopped at Epoch: {len(history.history['loss'])}")

# Plotting
plt.figure(figsize=(12, 5))

# Loss Plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss over Epochs')
plt.legend()

# AUC Plot
plt.subplot(1, 2, 2)
plt.plot(history.history['auc'], label='Train AUC')
plt.plot(history.history['val_auc'], label='Val AUC')
plt.title('AUC over Epochs')
plt.legend()

plt.show()